<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/food101_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

import torchvision
from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

In [2]:
# progress bar
from tqdm.auto import tqdm

#trains on gpu when available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

In [4]:
#convert the data to 128x128, flip it horizontally randomly, convert to PyTorch tensor
data_transforms = transforms.Compose([
    transforms.Resize(size = (64,64)),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.ToTensor()
])

In [5]:
#Food101 dataset : 101 classes, 750 training examples each and 250 test examples each, RGB channels

train_data = datasets.Food101(
    root = "data",
    download = True,
    transform = data_transforms,
    target_transform = None
)

100%|██████████| 4996278331/4996278331 [03:31<00:00, 23664265.28it/s]


Extracting data/food-101.tar.gz to data


In [6]:
test_data = datasets.Food101(
    root = "data",
    split = 'test',
    download = True,
    transform = data_transforms,
    target_transform = None
)

In [7]:
#makeing iterable batches

BATCH_SIZE = 25
train_dataloader = DataLoader(
    train_data,
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dataloader = DataLoader(
    test_data,
    batch_size = BATCH_SIZE,
    shuffle = False
)

In [8]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size, padding=padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

In [14]:
#Architecture from Food101 used in CNN Explainer website
#Contains 2 convolutionl blocks with conv->batchnorm->relu->conv->batchnorm->relu->maxpool->dropout in each block followed by a fully connected linear layer
class Food101(nn.Module):
    def __init__(self, num_classes=101):
        super(Food101, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = DepthwiseSeparableConv(32, 64, 3, padding=1)
        self.conv3 = DepthwiseSeparableConv(64, 128, 3, padding=1)
        self.conv4 = DepthwiseSeparableConv(128, 256, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)

        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

torch.manual_seed(42)
model_0 = Food101().to(device)


In [15]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),
                            lr=0.009)

In [17]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.to(device)
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1))

        # 3. reset gradient
        optimizer.zero_grad()

        # 4. backprop
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # loss and accuracy per epoch
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.to(device)
    model.eval()
    # inference context manager
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1)
            )

        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [18]:
#training loop

torch.manual_seed(42)

epochs = 100
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dataloader,
        model=model_0,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=test_dataloader,
        model=model_0,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 4.33463 | Train accuracy: 4.86%
Test loss: 3.95262 | Test accuracy: 11.17%

Epoch: 1
---------
Train loss: 3.89960 | Train accuracy: 11.35%
Test loss: 3.60143 | Test accuracy: 15.86%

Epoch: 2
---------
Train loss: 3.56663 | Train accuracy: 17.00%
Test loss: 3.36233 | Test accuracy: 20.19%

Epoch: 3
---------
Train loss: 3.33295 | Train accuracy: 21.19%
Test loss: 3.17890 | Test accuracy: 23.71%

Epoch: 4
---------
Train loss: 3.14615 | Train accuracy: 24.77%
Test loss: 3.09015 | Test accuracy: 25.31%

Epoch: 5
---------
Train loss: 2.99423 | Train accuracy: 27.60%
Test loss: 3.06736 | Test accuracy: 26.11%

Epoch: 6
---------
Train loss: 2.84890 | Train accuracy: 30.50%
Test loss: 3.04982 | Test accuracy: 26.76%

Epoch: 7
---------


KeyboardInterrupt: 